In [ ]:
#requirments if not installed

# !pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu118
# !pip install opencv-python
# !pip install pycocotools
# !pip install cython

In [1]:
#imports

import torch
import cv2
from yolact import Yolact
from collections import defaultdict
import torch.backends.cudnn as cudnn
from utils.augmentations import FastBaseTransform
from utils import timer
from data import cfg
from layers.output_utils import postprocess
from data import COLORS

In [2]:
if torch.cuda.is_available():
    cudnn.fastest = True
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
sc=0.45
top_k =15
color_cache = defaultdict(lambda: {})
display_masks=True;
display_text_bbx=True;

In [4]:
print('Loading model...', end='')
net = Yolact()
net.load_weights("model/yolact_base_54_800000.pth")
net.eval()
net.cuda()
print(' Done.')

Loading model...

C:\Users\deepa\anaconda3\envs\newtest\lib\site-packages\torch\jit\_recursive.py:260: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
C:\Users\deepa\anaconda3\envs\newtest\lib\site-packages\torch\jit\_recursive.py:260: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
C:\Users\deepa\anaconda3\envs\newtest\lib\site-packages\torch\jit\_recursive.py:260: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "


 Done.


In [5]:

def prep_display(dets_out, img, h, w, undo_transform=True, class_color=False, mask_alpha=0.45, fps_str=''):
    
    if undo_transform:
        img_numpy = undo_image_transformation(img, w, h)
        img_gpu = torch.Tensor(img_numpy).cuda()
    else:
        img_gpu = img / 255.0
        h, w, _ = img.shape
    
    with timer.env('Postprocess'):
        save = cfg.rescore_bbox
        cfg.rescore_bbox = True
        t = postprocess(dets_out, w, h, visualize_lincomb = False,
                                        crop_masks        = True,
                                        score_threshold   = 0)
        cfg.rescore_bbox = save
    

    with timer.env('Copy'):
        idx = t[1].argsort(0, descending=True)[:top_k]
        
        if cfg.eval_mask_branch:
            # Masks are drawn on the GPU, so don't copy
            masks = t[3][idx]
        classes, scores, boxes = [x[idx].cpu().numpy() for x in t[:3]]

    num_dets_to_consider = min(top_k, classes.shape[0])
    for j in range(num_dets_to_consider):
        if scores[j] < sc:
            num_dets_to_consider = j
            break

    def get_color(j, on_gpu=None):
        global color_cache
        color_idx = (classes[j] * 5 if class_color else j * 5) % len(COLORS)
        
        if on_gpu is not None and color_idx in color_cache[on_gpu]:
            return color_cache[on_gpu][color_idx]
        else:
            color = COLORS[color_idx]
            if not undo_transform:
                # The image might come in as RGB or BRG, depending
                color = (color[2], color[1], color[0])
            if on_gpu is not None:
                color = torch.Tensor(color).to(on_gpu).float() / 255.
                color_cache[on_gpu][color_idx] = color
            return color

    if display_masks and cfg.eval_mask_branch and num_dets_to_consider > 0:
        # After this, mask is of size [num_dets, h, w, 1]
        masks = masks[:num_dets_to_consider, :, :, None]
        colors = torch.cat([get_color(j, on_gpu=img_gpu.device.index).view(1, 1, 1, 3) for j in range(num_dets_to_consider)], dim=0)
        masks_color = masks.repeat(1, 1, 1, 3) * colors * mask_alpha

        inv_alph_masks = masks * (-mask_alpha) + 1

        masks_color_summand = masks_color[0]
        if num_dets_to_consider > 1:
            inv_alph_cumul = inv_alph_masks[:(num_dets_to_consider-1)].cumprod(dim=0)
            masks_color_cumul = masks_color[1:] * inv_alph_cumul
            masks_color_summand += masks_color_cumul.sum(dim=0)

        img_gpu = img_gpu * inv_alph_masks.prod(dim=0) + masks_color_summand

    img_numpy = (img_gpu * 255).byte().cpu().numpy()

    if num_dets_to_consider == 0:
        return img_numpy

    if display_text_bbx:
        for j in reversed(range(num_dets_to_consider)):
            x1, y1, x2, y2 = boxes[j, :]
            color = get_color(j)
            score = scores[j]

            if True:
                cv2.rectangle(img_numpy, (x1, y1), (x2, y2), color, 1)
                
                _class = cfg.dataset.class_names[classes[j]]
                text_str = '%s: %.2f' % (_class, score) if True else _class

                font_face = cv2.FONT_HERSHEY_DUPLEX
                font_scale = 0.6
                font_thickness = 1

                text_w, text_h = cv2.getTextSize(text_str, font_face, font_scale, font_thickness)[0]

                text_pt = (x1, y1 - 3)
                text_color = [255, 255, 255]

                cv2.rectangle(img_numpy, (x1, y1), (x1 + text_w, y1 - text_h - 4), color, -1)
                cv2.putText(img_numpy, text_str, text_pt, font_face, font_scale, text_color, font_thickness, cv2.LINE_AA)
    
    return img_numpy

In [6]:
def evaluateimg(img):
    
    frame = torch.from_numpy(img).cuda().float()

    batch = FastBaseTransform()(frame.unsqueeze(0))
    with torch.no_grad():
        preds = net(batch)

    res = prep_display(preds, frame, None, None, undo_transform=False)
    return res

In [7]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if ret:
        image = evaluateimg(frame)

    # Check if the frame was successfully captured
    if not ret:
        break

    # Display the captured frame on the screen
    cv2.imshow("Camera Output", image)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

AttributeError: module 'numpy' has no attribute 'int'